# Interpret-flair

##### Please note: this repository is not officially associated with [Flair](https://github.com/flairNLP/flair) nor with [Captum](https://github.com/pytorch/captum).
This notebook shows an attempt at integrating [Captum](https://github.com/pytorch/captum) with a custom trained [Flair text-classifier](https://github.com/flairNLP/flair).
As such, this approach should also be validated by outsiders.


The example model was trained on the [BBC dataset](https://www.kaggle.com/c/learn-ai-bbc/overview) and makes use of sentence-transformers' [xlm-r-100langs-bert-base-nli-mean-tokens model](https://huggingface.co/sentence-transformers/xlm-r-100langs-bert-base-nli-mean-tokens).
The model has the following results:
- F-score (micro) 0.964
- F-score (macro) 0.9626
- Accuracy 0.964

In [1]:
import numpy as np
from flair.models import TextClassifier
from flair.data import Sentence
from interpretation_package.flair_model_wrapper import ModelWrapper
from interpretation_package.interpret_flair import interpret_sentence, visualize_attributions
from captum.attr import LayerIntegratedGradients

In [2]:
model_path = "./model/output/best-model.pt"

We load the trained Flair classifier.

In [3]:
flair_model = TextClassifier.load(model_path)

2020-11-21 20:58:55,379 loading file ./model/output/best-model.pt


In order to make use of Captum's [LayerIntegratedGradients method](https://captum.ai/api/layer.html#layer-integrated-gradients) we had to rework Flair's forward function. This is handled by the wrapper.
The wrapper inherits functions of the Flair [text-classifier object](https://github.com/flairNLP/flair/blob/master/flair/models/text_classification_model.py) and allows us to calculate attributions with respect to a target class.

In [4]:
flair_model_wrapper = ModelWrapper(flair_model)

Let's check out the underlying XLMRoberta model.

In [5]:
print(flair_model_wrapper.model)

XLMRobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(250002, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0): RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropout): 

As described in the source code of [documentation of Captum](https://github.com/pytorch/captum/blob/master/captum/attr/_core/layer/layer_integrated_gradients.py):


"*Layer Integrated Gradients is a variant of Integrated Gradients that assigns
an importance score to layer inputs or outputs, depending on whether we
attribute to the former or to the latter one.*"


In this case, we are interested how the input embeddings of the model contribute to the output.

In [6]:
lig = LayerIntegratedGradients(flair_model_wrapper, flair_model_wrapper.model.embeddings)

To test this, let's take the two paragraphs of an article about business by the Economist.


[Which Japanese mogul will leave the biggest legacy?](https://www.economist.com/business/2020/11/07/which-japanese-mogul-will-leave-the-biggest-legacy)

In [7]:
sentence = """
In the 1990s, when a youthful Son Masayoshi, a Japanese entrepreneur, was pursuing acquisitions in his home country, he sought advice from a banker eight years his junior called Mikitani Hiroshi. They shared a lot in common: both had studied in America (Mr Son at the University of California, Berkeley, Mr Mikitani at Harvard Business School); they had a common interest in the internet; and they were both baseball mad. In the decades since, both men have blazed past a stifling corporate hierarchy to become two of Japan’s leading tech billionaires. 
Mr Mikitani, who says in an interview that he did not even know the word “entrepreneur” when he enrolled at Harvard, pioneered e-commerce in Japan via Rakuten, which is now a sprawling tech conglomerate worth $14bn. Mr Son’s SoftBank, after spectacular investments in early internet stocks, muscled into Japan’s telecoms industry. They have both invested heavily in Silicon Valley. They also each own baseball teams named after birds of prey; the SoftBank Hawks and the Rakuten Golden Eagles.
"""

For convience, let's check the label dictionary to see which is 'business'.


This can be useful if you have complex labels, or want to quickly reference labels used by the model.

In [8]:
print(flair_model_wrapper.label_dictionary.get_item_for_index(1))

target_label = flair_model_wrapper.label_dictionary.get_item_for_index(1)

business


We also create an empty list to store our attribitions results in order to visualize them using Captum.

In [9]:
visualization_list = []

Let's run the Layer Integrated Gradient method on the two paragraphs, and determine what drives the prediction.

As an additional note, the number of steps & the estimation method can have an impact on the attribution.

This [tutorial](https://colab.research.google.com/drive/1pgAbzUF2SzF0BdFtGpJbZPWUOhFxT2NZ#scrollTo=sO0Wr7j6TPOR) even uses 7000 steps!

In [10]:
readable_tokens, word_attributions, delta = interpret_sentence(flair_model_wrapper,
                                                                lig,
                                                                sentence,
                                                                target_label,
                                                                visualization_list,
                                                                n_steps=500,
                                                                estimation_method="gausslegendre",
                                                                internal_batch_size=3)

pred:  1 ( 1.00 ) , delta:  tensor([2.8829], dtype=torch.float64)


In [11]:
visualize_attributions(visualization_list)

The tokenizer used by your model will have an impact how the original text is displayed. 

We can also see the importance scores of each token.

In [14]:
word_scores = word_attributions.detach().numpy()

In [15]:
ordered_lig = [(readable_tokens[i], word_scores[i]) for i in np.argsort(word_scores)][::-1]

In [16]:
ordered_lig

[('investment', 0.6912556656584984),
 (',', 0.3798837938229196),
 ('In', 0.3476725938390601),
 ('.', 0.31968725095155986),
 ('Golden', 0.2094622213371851),
 ('roll', 0.15912006355488764),
 ('Eagle', 0.12119987913236946),
 ('each', 0.11796153579109278),
 ('have', 0.11295847290029525),
 ('interview', 0.06794168798818423),
 (',', 0.0591058601487673),
 ('s', 0.05599717840192191),
 ('kita', 0.04546959026524195),
 ('internet', 0.04273298068470459),
 ('even', 0.0398466819989191),
 ('internet', 0.03650298645706512),
 ('both', 0.035969422144733296),
 ('worth', 0.03288273963161129),
 ('billion', 0.03206918459566223),
 ('muscle', 0.028196057380916115),
 ('banker', 0.026940519313020748),
 ('ed', 0.024515105846343522),
 ('mera', 0.02168594978900262),
 ('after', 0.020827375280079875),
 ('Rak', 0.020516629732796308),
 ('uten', 0.019807849524593118),
 ('School', 0.019248880413689953),
 ('’', 0.01823743842859383),
 ('ed', 0.016867976719556504),
 ('Masa', 0.01644864465371571),
 ('California', 0.01628921

### Contributions & Suggestions

[Pull requests](https://github.com/robinvanschaik/interpret-flair/pulls) and [issues](https://github.com/robinvanschaik/interpret-flair/issues) are welcome! 


Check out this [discussion](https://github.com/flairNLP/flair/issues/1504) regarding explainable AI & Flair integration.

### Authors

* [Robin van Schaik](https://github.com/robinvanschaik)

### Acknowledgements

* [Flair](https://github.com/flairNLP/flair) for the text classification training framework.
* [Sentence transformers](https://github.com/UKPLab/sentence-transformers) for great sentence-level language models that can be used in Flair.
* [Huggingface](https://github.com/huggingface/transformers) for a large collection of language models that can be used in Flair.
* [Captum](https://github.com/pytorch/captum) for providing the model interpretation framework.
* This [tutorial](https://captum.ai/tutorials/Bert_SQUAD_Interpret) by the Captum team helped me to get started.
* This [discussion](https://github.com/pytorch/captum/issues/414) regarding Captum & XLM type models was also very insightful.